In [1]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde
from statsmodels.nonparametric.kde import KDEUnivariate
from decimal import *
import scipy as sp
from scipy.signal import butter, lfilter, freqz


%matplotlib inline
plt.ion()

In [20]:
#Copied and saved the data in a csv file and then read it here
df = pd.read_csv('/home/sbagherpour/Desktop/Data_Insight_1', header=None)
df




,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
5,1017122,8,10,10,8,7,10,9,7,1,4
6,1018099,1,1,1,1,2,10,3,1,1,2
7,1018561,2,1,2,1,2,1,3,1,1,2
8,1033078,2,1,1,1,2,1,1,1,5,2
9,1033078,4,2,1,1,2,1,2,1,1,2


In [64]:
#In the next step I want to check if there is any missing data
df.isnull().values.any()



False

In [115]:
#In this stage I want to see how imbalance are my class labales distributed
df[df.columns[10]].value_counts()
#2 for benign, 4 for malignant, so malignant ones are the minority and we want to give higher importance to them
#since the problem in hand has imbalance issue, I would use stratified cross validation

#I am going to use Decision trees, because they are good at handling imbalance and they are easy to explain to clinical doctors
#C4.5, Quinlan's next iteration. The new features (versus ID3) are: 
#(i) accepts both continuous and discrete features;  (iii) solves over-fitting problem by (very clever) bottom-up technique usually known as "pruning"; 
#and (iv) different weights can be applied the features that comprise the training data. Of these, the first three are very important--and i would suggest that any DT implementation you choose have all three. The fourth (differential weighting) is much less important
#Considering that imbalance highly affects trees I will use stratified cross validation

dfx = df.ix[:,1:9]
dfy=df.ix[:,10:10]

dfy.replace([2, 4], [1, 0], inplace=True)

X = dfx.as_matrix()
y=dfy.as_matrix()


In [120]:
import sklearn
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import KFold, cross_val_score

from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn import tree

#My data was string so I neeeded to convert them to be able to use decision tree
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for i in range(9):
    X[:,i] = le.fit_transform(X[:,i])

#Cross validation
crossvalidation = KFold(n=X.shape[0], n_folds=10, shuffle=True, random_state=1)

tree_classifier = tree.DecisionTreeClassifier(min_samples_split=10, min_samples_leaf=10,random_state=0)
tree_classifier.fit(X,y)

score = np.mean(cross_val_score(tree_classifier, X, y, scoring='f1', cv=crossvalidation, n_jobs=1))

print (score)


0.944226220349


In [ ]:
#If I had more time,I would have tried stratified Cross validation option
stratification = StratifiedKFold(n=X.shape[0], n_folds=10, shuffle=True, random_state=1)